<a href="https://colab.research.google.com/github/Gan4x4/CV-HSE2019/blob/master/video/exercise.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
#Detector
import sys 
!git clone https://github.com/ultralytics/yolov3.git
sys.path.insert(0,'/content/yolov3')
#MOT
!git clone https://github.com/abewley/sort.git

#Download video
!sudo wget https://yt-dl.org/downloads/latest/youtube-dl
!sudo chmod a+rx youtube-dl
!./youtube-dl -f22 -o video.mp4 https://www.youtube.com/watch?v=2RQrKP9a0XE


Cloning into 'yolov3'...
remote: Enumerating objects: 8350, done.
remote: Total 8350 (delta 0), reused 0 (delta 0), pack-reused 8350
Receiving objects: 100% (8350/8350), 6.77 MiB | 22.00 MiB/s, done.
Resolving deltas: 100% (5757/5757), done.
Cloning into 'sort'...
remote: Enumerating objects: 37, done.
remote: Counting objects: 100% (37/37), done.
remote: Compressing objects: 100% (19/19), done.
remote: Total 173 (delta 10), reused 32 (delta 6), pack-reused 136
Receiving objects: 100% (173/173), 1.20 MiB | 9.17 MiB/s, done.
Resolving deltas: 100% (56/56), done.
--2020-03-16 14:18:13--  https://yt-dl.org/downloads/latest/youtube-dl
Resolving yt-dl.org (yt-dl.org)... 95.143.172.170, 2001:1a50:11:0:5f:8f:acaa:177
Connecting to yt-dl.org (yt-dl.org)|95.143.172.170|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://yt-dl.org/downloads/2020.03.08/youtube-dl [following]
--2020-03-16 14:18:14--  https://yt-dl.org/downloads/2020.03.08/youtube-dl
Connecting to

In [0]:
from models import *
from utils.utils import *

device = torch.device("cuda:0")
print(device)
input_size = (416, 416)
model = Darknet("yolov3/cfg/yolov3.cfg", input_size ).to(device)

# Load weights
weights = 'yolov3/weights/yolov3.pt'
attempt_download(weights)

model.load_state_dict(torch.load(weights, map_location=device)['model'])
model.eval()

In [0]:
import queue
from torch.utils.data import Dataset
from PIL import Image
import time
import torch

def time_synchronized():
    torch.cuda.synchronize() if torch.cuda.is_available() else None
    return time.time()

class VideoDataset(Dataset):
  def __init__(self,transform):
    self.transform = transform
    self.buffer = queue.Queue()

  def __getitem__(self,i):
    sample = self.buffer.get()
    rgb_frame = cv2.cvtColor(sample, cv2.COLOR_BGR2RGB)
    im = Image.fromarray(rgb_frame)
    if self.transform:
        im = self.transform(im)
    return im
    
  def __len__(self):
    return self.buffer.qsize()


In [0]:
import cv2
from torch.utils.data import DataLoader
from torchvision import transforms
import time

transform=transforms.Compose([transforms.Resize((416,416)),
                              transforms.ToTensor() #Evil
                              ])
# ToTensor https://pytorch.org/docs/stable/torchvision/transforms.html#torchvision.transforms.ToTensor
#Converts a PIL Image or numpy.ndarray (H x W x C) in the range [0, 255] to a torch.FloatTensor of shape (C x H x W) 
#in the range [0.0, 1.0] if the PIL Image belongs to one of the modes (L, LA, P, I, F, RGB, YCbCr, RGBA, CMYK, 1) 
#or if the numpy.ndarray has dtype = np.uint8

dataset = VideoDataset(transform)
dataloader = DataLoader(dataset,  batch_size=1)

stream = cv2.VideoCapture("video.mp4")
fps = stream.get(cv2.CAP_PROP_FPS)
width  = int(stream.get(cv2.CAP_PROP_FRAME_WIDTH))   
height = int(stream.get(cv2.CAP_PROP_FRAME_HEIGHT))
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
out = cv2.VideoWriter('output.mp4',fourcc,fps, (width,height))
print ("FPS {} size {}x{}".format(fps,width,height))
MAX_FRAMES = 300
frames_read = 0
ratio_x = width/input_size[1]
ratio_y = height/input_size[0]
tm = time_synchronized()
cnn_tm = 0
nms_tm = 0
log_interval = 50
!nvidia-smi

while stream.isOpened() and frames_read < MAX_FRAMES:
  ret, frame = stream.read() 
  # How to read live stream?
  # - Read all frames withou delay
  # https://gist.github.com/allskyee/7749b9318e914ca45eb0a1000a81bf56
  if ret:
    dataset.buffer.put(frame)
    
    frames_read += 1
    # TODO Add motion detector

    # How to increase speed ?
    for images in dataloader:
      t1 = time_synchronized()
      images = images.to(device)

      output = model(images)
      detections = output[0]

      t2 = time_synchronized()
      cnn_tm += t2-t1
      results = non_max_suppression(detections)
      t3 =time_synchronized()
      nms_tm += t3 - t2
      
      # TODO add MOT
      if not results is None and not results[0] is None:
        for r in results[0]:
          x1 = r[0]*ratio_x
          x2 = r[2]*ratio_x
          y1 = r[1]*ratio_y
          y2 = r[3]*ratio_y
        cv2.rectangle(frame, (x1,y1), (x2,y2),color=(0, 255, 0), thickness=3)
    out.write(frame)

  if not frames_read % log_interval:
    print ("Processed {} frames. TOTAL_FPS: {:.1f}, YOLO_FPS: {:.1f}, NMS_FPS: {:.1f}" .format(frames_read,log_interval/(torch_utils.time_synchronized()-tm),log_interval/cnn_tm,log_interval/nms_tm))
    tm = time_synchronized()
    cnn_tm = 0
    nms_tm = 0
stream.release()
out.release()




FPS 29.97002997002997 size 1280x720
Sun Mar 15 18:36:49 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 440.59       Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   51C    P0    28W /  70W |   2231MiB / 15079MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
                                                                               
+-----------------------------------------------------------------------------+
| Processes:                                        